In [4]:
## helpful packages
import pandas as pd
import numpy as np
import random
import re
import recordlinkage
! pip install xlrd
! pip install openpyxl
 

## repeated printouts
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [5]:
# load data
addendum = pd.read_csv("FOIA_2021-F-05932_raw_data_combined.csv")
disclosure = pd.read_csv ("H-2A_Disclosure_Data_FY_combined.csv")

In [6]:
addendum.head()
disclosure.head()

,CASE_NUMBER,SECTION_NAME,SECTION_NUMBER,SECTION_DETAILS
0,H-300-19274-066174,Meal Provision,E.1,Employer will furnish free and convenient cook...
1,H-300-19274-066174,Job Requirements,B.6,Three (3) months experience with references re...
2,H-300-19274-066174,Daily Transportation,F.1,Living & laundry facilities available. Housing...
3,H-300-19274-066174,Job Requirements,B.6,Should the Employers worker’s compensation ins...
4,H-300-19274-066174,Daily Transportation,F.1,The employer shall provide transportation in t...


,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,TYPE_OF_EMPLOYER_APPLICATION,H2A_LABOR_CONTRACTOR,NATURE_OF_TEMPORARY_NEED,EMERGENCY_FILING,EMPLOYER_NAME,TRADE_NAME_DBA,...,ADDENDUM_B_HOUSING_ATTACHED,TOTAL_HOUSING_RECORDS,MEALS_PROVIDED,MEALS_CHARGED,MEAL_REIMBURSEMENT_MINIMUM,MEAL_REIMBURSEMENT_MAXIMUM,PHONE_TO_APPLY,EMAIL_TO_APPLY,WEBSITE_TO_APPLY,TOTAL_ADDENDUM_A_RECORDS
0,H-300-20265-835437,Determination Issued - Certification,2020/9/24,2020/9/30,Individual Employer,Y,Seasonal,N,Benito Martinez Jr.,NaN,...,N,1.0,N,$12.68,$12.68,$55.00,14073199505,benitomartinez863@gmail.com,NaN,0.0
1,H-300-20260-827678,Determination Issued - Certification,2020/9/16,2020/9/24,Association - Joint Employer,N,Seasonal,N,WESTERN RANGE ASSOCIATION,NaN,...,N,1.0,N,NaN,$12.68,$55.00,12085952226,applicants@westernrange.net,NaN,0.0
2,H-300-20260-827308,Determination Issued - Certification,2020/9/16,2020/9/24,Association - Agent,N,Seasonal,N,Allen Ranches LLC,NaN,...,N,1.0,N,NaN,$12.68,$55.00,13074722105,info@mpaswy.com,NaN,0.0
3,H-300-20260-826330,Determination Issued - Certification,2020/9/18,2020/9/30,Individual Employer,N,Seasonal,N,Steven Edge,NaN,...,N,1.0,N,NaN,$12.68,$55.00,18433407334,sdedge@sccoast.net,NaN,0.0
4,H-300-20259-823987,Determination Issued - Withdrawn,2020/9/18,2020/9/29,Individual Employer,N,Seasonal,N,Floyd Frey,NaN,...,N,1.0,N,NaN,$12.68,$55.00,14062650953,floydfrey@yahoo.com,NaN,0.0


In [45]:
#Data Cleaning
print(len(addendum.index))

spanish_words = ['COMPENSACIÓN', 'OCUPACIÓN', 'PAGO', 'TRANSPORTE', 'REGLAS', 'CONTRATO', 'HORAS'
                'TRABAJADORES', 'PARTE', 'TARIFAS', 'CUARTOS', 'TRABAJO', 'REGISTROS', 'ADICIONALES',
                'COMIDAS', 'EMPLEO', 'TERMINACIÓN', 'VIVIENDA', 'EMPLEADOR', 'ADICIONALES', 'FRECUENCIA',
                'PRODUCCIÓN', 'LIQUIDACIÓN', 'Comida', 'HERRAMIENTAS', 'PRUEBA', 'DIARIA', 'TARIFAS',
                'PARADO', 'ADICIONALES', 'COBERTURA', 'PROPORCIONADOS', 'PARTE', 'ABANDONO', 'DEDUCCIONES',
                'CAUSA', 'GARANTÍAS', 'DIVULGACIÓN', 'COMIDAS', 'POLÍTICA', 'DISCIPLINA', 'DESPIDOS', 'Derechos',
                'IMPOSIBILIDAD', 'DETALLES', 'DIVULGACIÓN', 'BLOQUEO', 'ABANDONO', 'GANANCIAS', 'COBERTURA', 'PEDIDOS',
                'PAGO', 'Oportunidad', 'Trabajo']

spanish_to_remove = "|".join(spanish_words)
print(spanish_to_remove)
temp = addendum[addendum.SECTION_NAME.str.contains(fr'{spanish_to_remove}', na = False, regex=True, case=False)]
print(temp.SECTION_NAME.sample(50))

print(len(temp.index))

98008
COMPENSACIÓN|OCUPACIÓN|PAGO|TRANSPORTE|REGLAS|CONTRATO|HORASTRABAJADORES|PARTE|TARIFAS|CUARTOS|TRABAJO|REGISTROS|ADICIONALES|COMIDAS|EMPLEO|TERMINACIÓN|VIVIENDA|EMPLEADOR|ADICIONALES|FRECUENCIA|PRODUCCIÓN|LIQUIDACIÓN|Comida|HERRAMIENTAS|PRUEBA|DIARIA|TARIFAS|PARADO|ADICIONALES|COBERTURA|PROPORCIONADOS|PARTE|ABANDONO|DEDUCCIONES|CAUSA|GARANTÍAS|DIVULGACIÓN|COMIDAS|POLÍTICA|DISCIPLINA|DESPIDOS|Derechos|IMPOSIBILIDAD|DETALLES|DIVULGACIÓN|BLOQUEO|ABANDONO|GANANCIAS|COBERTURA|PEDIDOS|PAGO|Oportunidad|Trabajo
63355       Job Duties - HORAS Y DECLARACIONES DE GANANCIAS
12386               Job Requirements - PRUEBA DE CIUDADANÍA
91824       Job Duties - HORAS Y DECLARACIONES DE GANANCIAS
49593                Job Duties - La Oportunidad de Trabajo
50110     Job Requirements - POLÍTICA DE DISCIPLINA PROG...
64537     Job Duties - HERRAMIENTAS Y EQUIPOS PROPORCION...
31366               Job Requirements - PRUEBA DE CIUDADANÍA
85253     Job Requirements - Work Rules and Disciplinary...
35973

In [ ]:
#Data Aggregation

In [26]:
# merging testing code (fully applicable after data cleaning and the necessary steps above)
## left join  
merged_data = pd.merge(disclosure,
                             addendum,
                             left_on = 'CASE_NUMBER',
                             right_on = 'CASE_NUMBER',
                             how = "left",
                             indicator = "case_merge_status")
merged_data.case_merge_status.value_counts()
merged_data.shape()


both          102204
left_only       3255
right_only         0
Name: case_merge_status, dtype: int64

In [28]:
# ## look at one that doesnt have job 
merged_data.loc[merged_data.case_merge_status == "left_only",
                  ['CASE_NUMBER','SECTION_DETAILS',"EMPLOYER_STATE"]].sample(50)

,CASE_NUMBER,SECTION_DETAILS,EMPLOYER_STATE
38583,H-300-20058-357459,NaN,TX
96989,H-300-20302-890861,NaN,ME
19507,H-300-20126-542381,NaN,MI
93300,H-300-20258-821922,NaN,TX
81069,H-300-19322-152241,NaN,AR
85434,H-300-19301-112140,NaN,LA
64082,H-300-20003-230979,NaN,ND
57368,H-300-20016-257359,NaN,ND
81276,H-300-19322-151764,NaN,ND
71448,H-300-19352-202796,NaN,LA


In [32]:
# merging testing code (fully applicable after data cleaning and the necessary steps above)
## inner join  
merged_data_inner = pd.merge(addendum,
                             disclosure,
                             left_on = 'CASE_NUMBER',
                             right_on = 'CASE_NUMBER',
                             how = "inner",
                             indicator = "case_merge_status")
merged_data_inner.case_merge_status.value_counts()


both          102204
left_only          0
right_only         0
Name: case_merge_status, dtype: int64

In [30]:
merged_data.loc[merged_data.case_merge_status == "both",
                  ['CASE_NUMBER','SECTION_DETAILS',"EMPLOYER_STATE"]].sample(50)

,CASE_NUMBER,SECTION_DETAILS,EMPLOYER_STATE
13625,H-300-20155-621632,The worker understands that if he abandons his...,FL
59604,H-300-20011-246424,Inbound transportation and subsistence fees wi...,ID
1880,H-300-20226-768140,Criminal Background Check: \nThe company may c...,MS
17523,H-300-20139-580472,El patron proveera gratuitamente un lugar con...,FL
71245,H-300-19352-204491,Outbound -\nCriteria for benefit are identical...,MI
103499,H-300-20338-937010,General Conditions Applicable to All Crops: Wo...,LA
52672,H-300-20024-273864,Should the Employers worker’s compensation ins...,ND
94416,H-300-20293-882161,Worker compensation insurance is provided.\nCa...,AL
63454,H-300-20004-232980,The work contract shall be terminated before t...,NC
53256,H-300-20023-270286,ADDENDUM C \n\nNew York State Specific Assuran...,NY


In [ ]:
# topic modeling code (fully applicable after data cleaning and the necessary steps above)
